In [23]:
import numpy as np
import matplotlib.pyplot as plt
plt.style.use('qualitative.mplstyle')
colors = plt.rcParams['axes.prop_cycle'].by_key()['color']

# Task 1

## 1D Gaussians: $\quad \mu_1 < \mu_2, \quad \sigma_1 = \sigma_2$

In [24]:
y1 = -1
y2 = +1
pys = {y1: 0.4, y2: 0.6}
assert pys[y1] + pys[y2] == 1.0, "Prior probabilities of the classes must sum to 1."
mus = {y1: 3, y2: 7}
assert mus[y1] < mus[y2], "Without loss of generality, we assume that mu2 > mu1."
sigma = 5

In [25]:
def py(y):
    return pys[y]

def pxy(x, y):
    return py(y) / (2*np.pi*sigma) * np.exp(-(x-mus[y])**2 / (2*sigma**2))

def get_threshold():
    return (2*sigma**2 * (np.log(py(y1)) - np.log(py(y2))) - mus[y1]**2 + mus[y2]**2)/(-2*mus[y1] + 2*mus[y2])

In [26]:
# Visualization limits
low, high = -10, 30
xs = np.arange(low, high, 0.01)

# Plot the p.d.f.
plt.plot(xs, pxy(xs, y1), c=colors[0], label="$p(x,y_1)$")
plt.plot(xs, pxy(xs, y2), c=colors[1], label="$p(x,y_2)$")

# Plot where we should classify as y1
plt.fill_betweenx([0, np.max(np.concatenate([pxy(xs, y1), pxy(xs, y2)]))],
                  low, get_threshold(),
                  alpha=0.3, color=colors[0], label="Classify as $y_1$")
# Plot where we should classify as y2
plt.fill_betweenx([0, np.max(np.concatenate([pxy(xs, y1), pxy(xs, y2)]))],
                  get_threshold(), high,
                  alpha=0.3, color=colors[1], label="Classify as $y_2$")

# Plot the true risk
optimal_risk = [min(a, b) for a, b in zip(pxy(xs, y1), pxy(xs, y2))]
plt.fill_between(xs, optimal_risk,
                 color=colors[3], alpha=0.3, label="True risk")

plt.xlim([low, high])
plt.ylim([0, np.max(np.concatenate([pxy(xs, y1), pxy(xs, y2)]))])

plt.xlabel('$x$')
plt.legend()

## 1D Gaussians: $\quad \mu_1 = \mu_2, \quad \sigma_1 > \sigma_2$

In [27]:
y1 = -1
y2 = +1
pys = {y1: 0.5, y2: 0.5}
assert pys[y1] + pys[y2] == 1.0, "Prior probabilities of the classes must sum to 1."
mu = 5
sigmas = {y1: 5, y2: 2}
assert sigmas[y1] > sigmas[y2]

In [28]:
def py(y):
    return pys[y]

def pxy(x, y):
    return py(y) / (2*np.pi*sigmas[y]) * np.exp(-(x-mu)**2 / (2*sigmas[y]**2))

def get_threshold():
    return (2*sigmas[y1]**2*sigmas[y2]**2)/(sigmas[y1]**2 - sigmas[y2]**2) * (-np.log(py(y1)) + np.log(py(y2)) + np.log(np.sqrt(2*np.pi*sigmas[y1]**2)) - np.log(np.sqrt(2*np.pi*sigmas[y2]**2)))

In [29]:
# Visualization limits
low, high = -10, 30
xs = np.arange(low, high, 0.01)

# Plot the p.d.f.
plt.plot(xs, pxy(xs, y1), c=colors[0], label="$p(x,y_1)$")
plt.plot(xs, pxy(xs, y2), c=colors[1], label="$p(x,y_2)$")

# Plot where we should classify as y1
plt.fill_betweenx([0, np.max(np.concatenate([pxy(xs, y1), pxy(xs, y2)]))],
                  low, mu-np.sqrt(get_threshold()),
                  alpha=0.3, color=colors[0])
plt.fill_betweenx([0, np.max(np.concatenate([pxy(xs, y1), pxy(xs, y2)]))],
                  mu+np.sqrt(get_threshold()), high,
                  alpha=0.3, color=colors[0], label="Classify as $y_1$")
# Plot where we should classify as y2
plt.fill_betweenx([0, np.max(np.concatenate([pxy(xs, y1), pxy(xs, y2)]))],
                  mu-np.sqrt(get_threshold()), mu+np.sqrt(get_threshold()),
                  alpha=0.3, color=colors[1],  label="Classify as $y_2$")

# Plot the true risk
optimal_risk = [min(a, b) for a, b in zip(pxy(xs, y1), pxy(xs, y2))]
plt.fill_between(xs, optimal_risk,
                 color=colors[3], alpha=0.3, label="True risk")

plt.xlim([low, high])
plt.ylim([0, np.max(np.concatenate([pxy(xs, y1), pxy(xs, y2)]))])
plt.xlabel('$x$')
plt.legend()


# Comments on Task 1
If $p(x,y)$ is known, then the smallest possible risk is
\begin{align*}
R^* &= \inf_{h\in\mathcal{Y}^\mathcal{X}} R(h) \\
&= \inf_{h\in\mathcal{Y}^\mathcal{X}} \sum_{x\in\mathcal{X}} \sum_{y\in\mathcal{Y}} p(x,y) \cdot l(y, h(x)) \\
&= \sum_{x\in\mathcal{X}} p(x) \cdot \inf_{y'\in\mathcal{Y}} \sum_{y\in\mathcal{Y}} p(y|x) \cdot l(y, y').
\end{align*}
Minimizer of the smallest possible risk is the Bayes inference rule
\begin{align*}
h^*(x) = \arg\min_{y'\in\mathcal{Y}} \sum_{y\in\mathcal{Y}} p(y|x) \cdot l(y, y').
\end{align*}
We only have two classes $\mathcal{Y} = \{-1, +1\}$. Hence, we can expand the sum
\begin{align*}
\sum_{y\in\mathcal{Y}} p(y|x) \cdot \llbracket y \neq y' \rrbracket = p(y_1|x) \cdot \overbrace{\llbracket y_1 \neq y' \rrbracket}^{0/1} + p(y_2|x) \cdot \overbrace{\llbracket y_2 \neq y' \rrbracket}^{1/0}
\end{align*}
and make predictions such that we minimize it. We always pay a penalty, either $p(y_1|x)$ or $p(y_2|x)$ and we should choose to pay the lower one. Hence, we can decide on the prediction by comparing when $p(y_1|x) > p(y_2|x)$ and vice versa.

The derivation itself is left to the reader. 

Note that it might be helpful to instead investigate $\ln{\frac{p(y_1|x)}{p(y_2|x)}}$.

# Comments on Task 2


We want to prove that the empirical risk
\begin{align*}
    R_{\mathcal{S}^l} &= \frac{1}{l} \sum_{i=1}^{l} \mathcal{L}(y_i, h(x_i))\\
\end{align*}
computed on a test set $\mathcal{S}^l$ is an unbiased estimator of the true risk.

To achieve this, we will need to use both assumptions in the "i.i.d." assumption. 

\begin{align*}
    &\mathbb{E}_{\mathcal{S}^l} \Bigg[
    \frac{1}{l} \sum_{i=1}^{l} \mathcal{L}(y_i, h(x_i))
    \Bigg] \\
    &= \mathbb{E}_{(x_1, y_1, x_2, y_2, \ldots, x_l, y_l) \sim p^l} \Bigg[
    \frac{1}{l} \sum_{i=1}^{l} \mathcal{L}(y_i, h(x_i))
    \Bigg] \\
    &= \int_{x_1} \sum_{y_1} \int_{x_2} \sum_{y_2} \ldots \int_{x_l} \sum_{y_l} p(x_1, y_1, x_2, y_2, \ldots, x_l, y_l) \Bigg[
    \frac{1}{l} \sum_{i=1}^{l} \mathcal{L}(y_i, h(x_i))
    \Bigg]\\
    &= \int_{x_1} \sum_{y_1} \int_{x_2} \sum_{y_2} \ldots \int_{x_l} \sum_{y_l} p(x_1, y_1) \cdot  p(x_2, y_2) \cdot \ldots \cdot p(x_l, y_l) \Bigg[
    \frac{1}{l} \sum_{i=1}^{l} \mathcal{L}(y_i, h(x_i))
    \Bigg]\\
    &= \frac{1}{l} \Bigg(\int_{x_1} \sum_{y_1} p(x_1, y_1) \cdot \mathcal{L}(y_1, h(x_1)) + 
    \int_{x_2} \sum_{y_2} p(x_2, y_2) \cdot \mathcal{L}(y_2, h(x_2)) + \ldots + \int_{x_l} \sum_{y_l} p(x_l, y_l) \cdot \mathcal{L}(y_l, h(x_l))
    \Bigg)\\
    \ldots
\end{align*}

Given this simplification which we get by using the "independence" assumption, we should be able to easily obtain the result.

# Task 3

In [30]:
xs = np.arange(1, 10**5, 10)

for delta, color in zip([0.1, 0.05, 0.01], colors):
    plt.plot(xs, [np.sqrt((np.log(2)-np.log(delta))/(2*l)) for l in xs], c=color, label=f"$\delta={delta}$")

plt.xscale('log')
plt.xlabel("Number of Samples $l$")
plt.ylabel("$\epsilon$")
plt.legend()


In [31]:
epsilon = 0.01
delta = 1 - 0.95
(np.log(2)-np.log(delta))/(2*epsilon**2)

18444.397270569676

## Comments on Task 3

Take the Hoeffding inequality
\begin{align*}
    P(|\hat\mu - \mu| \geq \epsilon) \leq 2e^{-\frac{2l\epsilon^2}{(b-a)^2}}.
\end{align*}

Realizing that the assignment asks about the complementary event, we write
\begin{align*}
P(|\hat\mu - \mu| < \epsilon) &= 1 - P(|\hat\mu - \mu| \geq \epsilon) \geq 1 - 2e^{-\frac{2l\epsilon^2}{(b-a)^2}} = \gamma = 1-\delta.
\end{align*}

Deriving the formulas is then simple algebra.